In [13]:
import pandas as pd
import geopandas as gpd
from datetime import timedelta

import datetime
import altair as alt

import topojson

from zipfile import ZipFile


# INPUT

## shapes

In [30]:
bezirke_layer=gpd.read_file(r"..\\..\\berlin_vzaehlung\\shapes\\BerlinerBezirke.geojson")

bezirke_layer=bezirke_layer[['name','geometry']].copy()
bezirke_layer=bezirke_layer[bezirke_layer.name.notnull()].copy()
bezirke_layer.geometry=bezirke_layer.buffer(0.00001)
bezirke_layer.name=bezirke_layer.name.str.replace('ö','oe')


#krs.geometry= krs.buffer(0) #falls repair geometry notwendig\n",
bezirke_layer_tj = topojson.Topology(bezirke_layer, prequantize=False, topology=True)
bezirke_layer_tj_sim = bezirke_layer_tj.toposimplify(0.002)
bezirke_layer = bezirke_layer_tj_sim.to_gdf()
bezirke_layer.crs='epsg:4326'

bezirke=alt.Chart(bezirke_layer).mark_geoshape(stroke='white',strokeWidth=0.5).encode()
bezirke

alt.Chart(...)

## Standorte

In [31]:
standort_daten=pd.read_excel(r"C:\Users\Simon\Analysen\eco-counter\Berlin\Gesamtdatei_Stundenwerte_2014_2018.xlsx", sheet_name='Standortdaten')

standort_daten['Beschreibung - Fahrtrichtung']=standort_daten['Beschreibung - Fahrtrichtung'].str.replace('Nord','').str.replace('Süd','').str.replace('Ost','').str.replace('West','')
standort_daten_=standort_daten.drop_duplicates('Beschreibung - Fahrtrichtung')[['Beschreibung - Fahrtrichtung','Breitengrad','Längengrad']]

standort_daten_['Beschreibung - Fahrtrichtung']=standort_daten_['Beschreibung - Fahrtrichtung'].str.replace(' ','').str.replace('-','').str.replace('ü','ue').str.replace('ß','ss')
standort_daten_=standort_daten_.rename(columns={'Beschreibung - Fahrtrichtung':'location_parent'})
standort_daten_=standort_daten_[standort_daten_.location_parent!='Yorkstrasse'].copy()

In [32]:
standort_daten_.head()

,location_parent,Breitengrad,Längengrad
0,SchwedterSteg,52.548836,13.400367
1,Jannowitzbruecke,52.513932,13.417835
3,Prinzregentenstrasse,52.488136,13.333120
4,Yorckstrasse,52.491938,13.373472
6,Monumentenstrasse,52.488117,13.369785


In [125]:
single_nearest = alt.selection_single(on='mouseover', empty='none')

points=alt.Chart(standort_daten_).mark_circle(
    color='black',
    stroke='white',strokeWidth=0.0
).encode(
    longitude='Längengrad:Q',
    latitude='Breitengrad:Q',
    size=alt.condition(single_nearest, alt.value(100), alt.value(40)),
   # text=alt.Text('Beschreibung - Fahrtrichtung:O')
).add_selection(
    single_nearest
)

bezirke + points

alt.LayerChart(...)

## Radzähldaten

In [27]:
day_data= pd.read_csv('out_all_day_til2004.csv', encoding='latin1')

day_data['location_parent']=day_data.location.str.split('_').str[0]

day_data.timestamp=pd.to_datetime(day_data.timestamp)

day_data['weekday']=day_data.timestamp.dt.weekday
day_data['date']=day_data.timestamp.dt.date
day_data['weeknr']=day_data.timestamp.dt.week
day_data['month']=day_data.timestamp.dt.month
day_data['year']=day_data.timestamp.dt.year
day_data['weeknr']=day_data.timestamp.dt.week
day_data.head()

,name,location,volume,timestamp,location_parent,weekday,date,weeknr,month,year
0,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,1901,2015-04-01,Jannowitzbrücke,2,2015-04-01,14,4,2015
1,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,1887,2015-04-02,Jannowitzbrücke,3,2015-04-02,14,4,2015
2,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,970,2015-04-03,Jannowitzbrücke,4,2015-04-03,14,4,2015
3,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,1488,2015-04-04,Jannowitzbrücke,5,2015-04-04,14,4,2015
4,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,1118,2015-04-05,Jannowitzbrücke,6,2015-04-05,14,4,2015


## Wetterdaten
* https://www.dwd.de/DE/leistungen/klimadatendeutschland/klarchivstunden.html

In [92]:
zip_file = ZipFile(r'wetter/stundenwerte_TU_00403_hist.zip')
data_temp_hist= pd.read_csv(zip_file.open('produkt_tu_stunde_19500101_20181231_00403.txt'), sep=';')

data_temp_hist['datetime'] = pd.to_datetime(data_temp_hist['MESS_DATUM'], format='%Y%m%d%H')

data_temp_hist = data_temp_hist[data_temp_hist['TT_TU']!=-999.].copy()

In [93]:
zip_file = ZipFile(r'wetter/stundenwerte_TU_00403_akt_new.zip')
data_temp= pd.read_csv(zip_file.open('produkt_tu_stunde_20181017_20200418_00403.txt'), sep=';')

data_temp['datetime'] = pd.to_datetime(data_temp['MESS_DATUM'], format='%Y%m%d%H')

data_temp = data_temp[data_temp['TT_TU']!=-999.].copy()

In [94]:
data_temp_all=data_temp_hist.append(data_temp)
data_temp_all=data_temp_all.drop_duplicates().copy()
data_temp_all.head()

,STATIONS_ID,MESS_DATUM,QN_9,TT_TU,RF_TU,eor,datetime
0,403,2002010100,3,-5.5,88.0,eor,2002-01-01 00:00:00
1,403,2002010103,3,-3.7,86.0,eor,2002-01-01 03:00:00
2,403,2002010106,3,-2.0,96.0,eor,2002-01-01 06:00:00
3,403,2002010109,3,-0.4,92.0,eor,2002-01-01 09:00:00
4,403,2002010112,3,0.6,86.0,eor,2002-01-01 12:00:00


# 1_proccesing
* raddaten:
    * groupby day starting march 2020
        * get weekly avarage: week 10 vs all others
    * groupby month starting 2015
* wetterdaten:
    * groupby day starting march 2020
    * groupby month starting 2015

In [63]:
day_data_grp=day_data.groupby(['location_parent','date']).volume.sum().reset_index()
day_data_grp=day_data_grp[day_data_grp.date>datetime.date(2020,3,1)]

In [47]:
week_data=day_data[(day_data.year==2020) & (day_data.weeknr>9) & (day_data.weekday<4)].groupby(['weeknr','location_parent','date']).volume.sum().reset_index()
week_data = week_data.groupby(['weeknr','location_parent']).volume.mean().reset_index()

In [48]:
week_data_week10=week_data[week_data.weeknr==10]
week_data_week10.head()

,weeknr,location_parent,volume
0,10,Alberichstraße,450.50
1,10,BerlinerStraße,5679.25
2,10,Breitenbachplatz,1932.75
3,10,FrankfurterAllee,3639.50
4,10,Invalidenstraße,4262.00


In [51]:
week_data_afterweek10=week_data[week_data.weeknr>10].copy()
week_data_afterweek10.head()

,weeknr,location_parent,volume
17,11,Alberichstraße,415.75
18,11,BerlinerStraße,5031.00
19,11,Breitenbachplatz,1726.50
20,11,FrankfurterAllee,3376.25
21,11,Invalidenstraße,3876.75


In [ ]:
# defining first and last week day for plotting

In [ ]:
day_data[day_data.year==2020]

In [55]:
week_data_afterweek10['xmin']=week_data_afterweek10.weeknr.apply(lambda x: day_data[day_data.year==2020].groupby('weeknr').date.min()[x])

In [56]:
week_data_afterweek10['xmax']=week_data_afterweek10.weeknr.apply(lambda x: day_data[day_data.year==2020].groupby('weeknr').date.min()[x]+ timedelta(days=4))

In [58]:
week_data_afterweek10.head()

,weeknr,location_parent,volume,xmin,xmax
17,11,Alberichstraße,415.75,2020-03-09,2020-03-13
18,11,BerlinerStraße,5031.00,2020-03-09,2020-03-13
19,11,Breitenbachplatz,1726.50,2020-03-09,2020-03-13
20,11,FrankfurterAllee,3376.25,2020-03-09,2020-03-13
21,11,Invalidenstraße,3876.75,2020-03-09,2020-03-13


In [450]:
#first_week_daily_avg_after10['xmax']=first_week_daily_avg_after10.xmax.apply(lambda x: datetime.date(2020,3,31) if x>datetime.date(2020,3,31) else x)

In [64]:
import altair as alt

source = day_data_grp.copy()

source.date=source.date.astype(str)

source2 = week_data_afterweek10.copy()
source2.xmin=source2.xmin.astype(str)
source2.xmax=source2.xmax.astype(str)


selection = alt.selection_multi(fields=['location_parent'])


chart=alt.Chart(source).mark_bar().encode(
    x='date:T',
    y='sum(volume):Q',
    color=alt.Color('location_parent:N', legend=None),
).transform_filter(
    selection
)


holine = alt.Chart(week_data_week10).mark_rule().encode(
    y='volume:Q'
).transform_filter(
    selection
)

holine2 = alt.Chart(source2).mark_rule(color='red').encode(
    y='volume:Q',
    x="xmin:T",
    x2="xmax:T"
).transform_filter(
    selection
)


## interactive legend
legend = alt.Chart(source).mark_rect().encode(
        y=alt.Y('location_parent:N'),#  sort=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]),
        color=alt.condition(selection,
                    alt.Color('location_parent:N', legend=None),
                    alt.value('lightgray'))
    ).add_selection(
    selection
).properties(
        width=20,
    )



chart+holine +holine2 | legend

alt.HConcatChart(...)

In [65]:
data_temp_all

,STATIONS_ID,MESS_DATUM,QN_9,TT_TU,RF_TU,eor,datetime
0,403,2002010100,3,-5.5,88.0,eor,2002-01-01 00:00:00
1,403,2002010103,3,-3.7,86.0,eor,2002-01-01 03:00:00
2,403,2002010106,3,-2.0,96.0,eor,2002-01-01 06:00:00
3,403,2002010109,3,-0.4,92.0,eor,2002-01-01 09:00:00
4,403,2002010112,3,0.6,86.0,eor,2002-01-01 12:00:00
...,...,...,...,...,...,...,...
13193,403,2020041819,1,12.4,31.0,eor,2020-04-18 19:00:00
13194,403,2020041820,1,10.7,32.0,eor,2020-04-18 20:00:00
13195,403,2020041821,1,9.1,36.0,eor,2020-04-18 21:00:00
13196,403,2020041822,1,9.0,36.0,eor,2020-04-18 22:00:00


In [101]:
data_temp_all['day']=data_temp_all.datetime.dt.day
data_temp_all['month']=data_temp_all.datetime.dt.month
data_temp_all['date']=data_temp_all.datetime.dt.date

In [97]:
data_temp_all_hist_avg=data_temp_all.groupby(['month','day']).TT_TU.mean().reset_index()

for i,row in data_temp_all_hist_avg.iterrows():
    data_temp_all_hist_avg.at[i,'date']=datetime.date(2020,int(row.month),int(row.day)) 
    
data_temp_all_hist_avg['type_']='historic'
data_temp_all_hist_avg.head()

,month,day,TT_TU,date,type_
0,1,1,2.115227,2020-01-01,historic
1,1,2,1.800683,2020-01-02,historic
2,1,3,0.718223,2020-01-03,historic
3,1,4,1.051818,2020-01-04,historic
4,1,5,0.135253,2020-01-05,historic


In [105]:
data_temp_current=data_temp_all.groupby(['date']).TT_TU.mean().reset_index()
data_temp_current=data_temp_current[data_temp_current.date>=datetime.date(2020,3,1) ]
data_temp_current['type_']='current'

In [118]:
data_temp_current.date.unique()

array([datetime.date(2020, 3, 1), datetime.date(2020, 3, 2),
       datetime.date(2020, 3, 3), datetime.date(2020, 3, 4),
       datetime.date(2020, 3, 5), datetime.date(2020, 3, 6),
       datetime.date(2020, 3, 7), datetime.date(2020, 3, 8),
       datetime.date(2020, 3, 9), datetime.date(2020, 3, 10),
       datetime.date(2020, 3, 11), datetime.date(2020, 3, 12),
       datetime.date(2020, 3, 13), datetime.date(2020, 3, 14),
       datetime.date(2020, 3, 15), datetime.date(2020, 3, 16),
       datetime.date(2020, 3, 17), datetime.date(2020, 3, 18),
       datetime.date(2020, 3, 19), datetime.date(2020, 3, 20),
       datetime.date(2020, 3, 21), datetime.date(2020, 3, 22),
       datetime.date(2020, 3, 23), datetime.date(2020, 3, 24),
       datetime.date(2020, 3, 25), datetime.date(2020, 3, 26),
       datetime.date(2020, 3, 27), datetime.date(2020, 3, 28),
       datetime.date(2020, 3, 29), datetime.date(2020, 3, 30),
       datetime.date(2020, 3, 31), datetime.date(2020, 4, 1),
  

In [120]:
data_temp_all_hist_avg=data_temp_all_hist_avg[data_temp_all_hist_avg.date.isin(list(data_temp_current.date.unique()))].copy()

In [121]:
source_temp=data_temp_current.append(data_temp_all_hist_avg[['date','TT_TU','type_']])

In [122]:
source_temp.head()

,date,TT_TU,type_
6634,2020-03-01,6.945833,current
6635,2020-03-02,5.795833,current
6636,2020-03-03,4.508333,current
6637,2020-03-04,4.058333,current
6638,2020-03-05,4.858333,current


In [123]:
source_temp.date=source_temp.date.astype(str)

domain = ['current', 'historic']
range_ = ['#507BAA', 'gray']

chart_temp=alt.Chart(source_temp).mark_line().encode(
    x='date:T',
    y='TT_TU:Q',
    color=alt.Color('type_:N', scale=alt.Scale(domain=domain, range=range_)),
).properties(
        height=100,
    )


(chart_temp ).resolve_legend(color="independent").resolve_scale(x='shared',color="independent") 


alt.Chart(...)

---

In [138]:
#input
source = day_data_grp.copy()

source.date=source.date.astype(str)
source['location_parent']=source['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')

source1 = week_data_week10.copy()
source1['location_parent']=source1['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')

source2 = week_data_afterweek10.copy()
source2.xmin=source2.xmin.astype(str)
source2.xmax=source2.xmax.astype(str)
source2['location_parent']=source2['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')

source_temp.date=source_temp.date.astype(str)

#selections
selection = alt.selection_single(fields=['location_parent'], init={'location_parent': 'MariendorferDamm'})
single_nearest = alt.selection_single(on='mouseover', empty='none')

#style
color_bars=alt.Color('location_parent:N', legend=None)

#### BARS

chart=alt.Chart(source).mark_bar().encode(
    x=alt.X('date:T', axis=alt.Axis(orient='top')),
    y='volume:Q',
    color=color_bars,
    tooltip='location_parent'
).transform_filter(
    selection
)


holine = alt.Chart(source1).mark_rule().encode(
    y='volume:Q'
).transform_filter(
    selection
)

holine2 = alt.Chart(source2).mark_rule(color='red').encode(
    y='volume:Q',
    x="xmin:T",
    x2="xmax:T"
).transform_filter(
    selection
)

#### TEMP

domain = ['current', 'historic']
range_ = ['#507BAA', 'gray']


chart_temp=alt.Chart(source_temp).mark_line().encode(
    x='date:T',
    y='TT_TU:Q',
    color=alt.Color('type_:N', scale=alt.Scale(domain=domain, range=range_)),
).properties(
        height=100,
    )

#### MAP
bezirke=alt.Chart(bezirke_layer).mark_geoshape(stroke='white',strokeWidth=0.5, color='lightgray').encode()


points=alt.Chart(standort_daten_).mark_circle(
    color='black',
    stroke='white',strokeWidth=0.0
).encode(
    longitude='Längengrad:Q',
    latitude='Breitengrad:Q',
    size=alt.condition(single_nearest, alt.value(150), alt.value(70)),
   # text=alt.Text('Beschreibung - Fahrtrichtung:O')
).add_selection(
    selection,single_nearest
)

text=alt.Chart(standort_daten_).mark_text(
    color='black',dy=-10, strokeWidth=0
).encode(
    longitude='Längengrad:Q',
    latitude='Breitengrad:Q',
    text=alt.Text('location_parent:O'),
    fillOpacity=alt.condition(single_nearest, alt.value(1), alt.value(0)),
)

text_source=alt.Chart(standort_daten_).mark_text(
    color='black', dx=-150, dy=-140
).encode(
    text=alt.Text('location_parent:O'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0)),
).add_selection(
    selection#,single_nearest
)


final_chart=alt.hconcat(
    ((chart+holine +holine2 + text_source)& chart_temp ).resolve_legend(color="independent").resolve_scale(x='shared',color="independent"),
    (bezirke + points+ text)
).configure_view(stroke=None)

final_chart

alt.HConcatChart(...)

In [505]:
final_chart.save('out_count_temp_corona.html')

In [559]:
day_data= pd.read_csv('out_all_day_til2004.csv', encoding='latin1')

day_data['location_parent']=day_data.location.str.split('_').str[0]

day_data.timestamp=pd.to_datetime(day_data.timestamp)

day_data['weekday']=day_data.timestamp.dt.weekday
day_data['date']=day_data.timestamp.dt.date
day_data['weeknr']=day_data.timestamp.dt.week
day_data['month']=day_data.timestamp.dt.month
day_data['year']=day_data.timestamp.dt.year

In [560]:
day_data['location_parent'].unique()

array([' Jannowitzbrücke', ' Oberbaumbrücke', ' Alberichstraße',
       ' FrankfurterAllee', ' PaulundPaulaUferweg', ' BerlinerStraße',
       ' Maybachufer', ' Yorckstraße', ' Monumentenstraße',
       ' Prinzregentenstraße', ' Breitenbachplatz', ' MariendorferDamm',
       ' Klosterstraße', ' Markstraße', ' SchwedterSteg',
       ' Invalidenstraße', ' Kaisersteg'], dtype=object)

In [561]:
#day_data=day_data[day_data.location_parent==' Jannowitzbrücke']

In [562]:
day_data

,name,location,volume,timestamp,location_parent,weekday,date,weeknr,month,year
0,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,1901,2015-04-01,Jannowitzbrücke,2,2015-04-01,14,4,2015
1,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,1887,2015-04-02,Jannowitzbrücke,3,2015-04-02,14,4,2015
2,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,970,2015-04-03,Jannowitzbrücke,4,2015-04-03,14,4,2015
3,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,1488,2015-04-04,Jannowitzbrücke,5,2015-04-04,14,4,2015
4,02-MI-JAN-N,Jannowitzbrücke_Alexanderplatz,1118,2015-04-05,Jannowitzbrücke,6,2015-04-05,14,4,2015
...,...,...,...,...,...,...,...,...,...,...
43762,23-TK-KAI,Kaisersteg_Beide,933,2020-04-14,Kaisersteg,1,2020-04-14,16,4,2020
43763,23-TK-KAI,Kaisersteg_Beide,1058,2020-04-15,Kaisersteg,2,2020-04-15,16,4,2020
43764,23-TK-KAI,Kaisersteg_Beide,1346,2020-04-16,Kaisersteg,3,2020-04-16,16,4,2020
43765,23-TK-KAI,Kaisersteg_Beide,1339,2020-04-17,Kaisersteg,4,2020-04-17,16,4,2020


In [563]:
day_data.groupby(['month','year','location_parent']).volume.mean().reset_index()

,month,year,location_parent,volume
0,1,2013,SchwedterSteg,709.129032
1,1,2014,SchwedterSteg,901.000000
2,1,2015,SchwedterSteg,1144.709677
3,1,2016,Alberichstraße,129.483871
4,1,2016,Invalidenstraße,663.112903
...,...,...,...,...
966,12,2019,Oberbaumbrücke,0.000000
967,12,2019,PaulundPaulaUferweg,1563.935484
968,12,2019,Prinzregentenstraße,838.774194
969,12,2019,SchwedterSteg,1191.419355


In [565]:
source=day_data.groupby(['month','year','location_parent']).volume.mean().reset_index()

source=source[source.year>2017]

alt.Chart(source).mark_line().encode(
    x='month:N',
    y='volume:Q',
    color=alt.Color('year:O'),
    detail='location_parent'
).properties(width=500)

alt.Chart(...)

In [570]:
source=day_data.groupby(['month','year','location_parent']).volume.mean().reset_index()

source=source[source.year>2015]


selection = alt.selection_multi(fields=['location_parent'])#, init={'weekday': 'Monday'})


chart=alt.Chart(source).mark_line().encode(
    x='month:N',
    y='volume:Q',
    color=alt.Color('year:O'),
    detail='location_parent'
).transform_filter(
    selection
).properties(width=500)




legend = alt.Chart(source).mark_rect().encode(
        y=alt.Y('location_parent:O'),#  sort=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]),
         color=alt.condition(selection,
                    alt.value('gray'),
                    alt.value('lightgray'))
    
        #color=alt.condition(selection,
        #            alt.value('blue'),#, legend=None),
        #            alt.value('lightgray'))
    ).add_selection(
    selection
).properties(
        width=20,
    )

chart | legend

alt.HConcatChart(...)

In [589]:
source=day_data.groupby(['month','year','location_parent']).volume.mean().reset_index()

source=source[source.year>2015]
source['location_parent']=source['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')

selection = alt.selection_multi(fields=['location_parent'])#, init={'weekday': 'Monday'})


chart=alt.Chart(source).mark_bar().encode(

    x=alt.X('year:N', title=''),
    y=alt.Y('volume:Q'),
    color=alt.Color('year:O'),
    detail='location_parent',
    facet=alt.Facet('month:O', spacing =5)
).transform_filter(
    selection
).properties(width=50)




legend = alt.Chart(source).mark_rect().encode(
        y=alt.Y('location_parent:O'),#  sort=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]),
         color=alt.condition(selection,
                    alt.value('gray'),
                    alt.value('lightgray'))
    
        #color=alt.condition(selection,
        #            alt.value('blue'),#, legend=None),
        #            alt.value('lightgray'))
    ).add_selection(
    selection
).properties(
        width=20,
    )

chart | legend

alt.HConcatChart(...)

-----------------------

In [642]:
import altair as alt

source = grp.copy()

source['location_parent']=source['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')

source['weekday_name']=pd.to_datetime(source.date).dt.weekday_name

source.date=source.date.astype(str)

color_bars=alt.Color('location_parent:N', legend=None)



#first_week_daily_avg_all_after10=data[(data.weeknr>10) & (data.weekday<4)].groupby(['weeknr','location_parent','date']).volume.sum().reset_index()

#first_week_daily_avg_after10 = first_week_daily_avg_all_after10.groupby(['weeknr','location_parent']).volume.mean().reset_index()
#first_week_daily_avg_after10

source2 = first_week_daily_avg_after10.copy()
source2.xmin=source2.xmin.astype(str)
source2.xmax=source2.xmax.astype(str)
source2['location_parent']=source2['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')


source1 = first_week_daily_avg.copy()
source1['location_parent']=source1['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')



source_temp= date_temp_day.copy()

source_temp['type_']='current'
data_temp_hist_day_march['type_']='historic'
source_temp=source_temp.append(data_temp_hist_day_march[['date','TT_TU','type_']])
#source_temp=source_temp[source_temp.date<'2020-03-31']

source_temp.date=source_temp.date.astype(str)

selection = alt.selection_single(fields=['location_parent'], empty='none', init={'location_parent': 'MariendorferDamm'})

single_nearest = alt.selection_single(on='mouseover', empty='none')

chart=alt.Chart(source).mark_bar().encode(
    x=alt.X('date:T', axis=alt.Axis(orient='top')),
    y='volume:Q',
    color=color_bars,
    #tooltip='location_parent'
    tooltip=['location_parent','volume','weekday_name'] #,'date:N'
).transform_filter(
    selection
).add_selection(
    selection
)


holine = alt.Chart(source1).mark_rule(strokeDash=[5,5], color='gray').encode(
    y='volume:Q'
).transform_filter(
    selection
)

holine2 = alt.Chart(source2).mark_rule(color='red').encode(
    y='volume:Q',
    #x="xmin:T",
    x=alt.X('xmin:T', axis=None),
    x2="xmax:T"
    #x2=alt.X2('xmax:T', axis=None),
).transform_filter(
    selection
)



domain = ['current', 'historic']
range_ = ['#507BAA', 'gray']


chart_temp=alt.Chart(source_temp).mark_line().encode(
    x='date:T',
    y='TT_TU:Q',
    color=alt.Color('type_:N', scale=alt.Scale(domain=domain, range=range_)),
).properties(
        height=100,
    )

#####################

bezirke=alt.Chart(bezirke_layer).mark_geoshape(stroke='white',strokeWidth=0.5, color='lightgray').encode()


points=alt.Chart(standort_daten_).mark_circle(
    color='black',
    stroke='white',strokeWidth=0.0
).encode(
    longitude='Längengrad:Q',
    latitude='Breitengrad:Q',
    size=alt.condition(single_nearest, alt.value(150), alt.value(70)),
   # text=alt.Text('Beschreibung - Fahrtrichtung:O')
).add_selection(
    selection,single_nearest
).properties(
        height=450,
        width=450,
    )

text=alt.Chart(standort_daten_).mark_text(
    color='black',dy=-10, strokeWidth=0
).encode(
    longitude='Längengrad:Q',
    latitude='Breitengrad:Q',
    text=alt.Text('location_parent:O'),
    fillOpacity=alt.condition(single_nearest, alt.value(1), alt.value(0)),
)

text_source=alt.Chart(standort_daten_).mark_text(
    color='black', dx=-150, dy=-140
).encode(
    text=alt.Text('location_parent:O'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0)),
)#.add_selection(
#    selection#,single_nearest
#)



source_bar=day_data.groupby(['month','year','location_parent']).volume.mean().reset_index()

source_bar=source_bar[source_bar.year>2015]
source_bar['location_parent']=source_bar['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')

#selection = alt.selection_multi(fields=['location_parent'])#, init={'weekday': 'Monday'})


chart_bar=alt.Chart(source_bar).mark_bar().encode(

    x=alt.X('year:N', title=''),
    y=alt.Y('volume:Q'),
    color=alt.Color('year:O', legend=None),
    detail='location_parent',
    facet=alt.Facet('month:O', spacing =5),
    tooltip=['location_parent','volume']
).transform_filter(
    selection
).add_selection(
    selection
).properties(width=50)


#count_chart= (chart+holine +holine2)
#main_chart=(count_chart+chart_temp).resolve_scale(y='independent')
#(main_chart| legend)


#(((chart+holine +holine2)+ chart_temp )| legend).resolve_scale(
#    y='independent'
#)

#(((chart+holine +holine2)& chart_temp+chart_temp_reg+chart_temp_hist )| legend)

#((chart+holine +holine2)& chart_temp+chart_temp_reg+chart_temp_hist ).resolve_scale(x='shared')| legend
#((chart+holine +holine2 + text_source)& chart_temp ).resolve_legend(color="independent").resolve_scale(x='shared',color="independent") | (bezirke + points+ text)


#((chart+holine +holine2 + text_source)& chart_temp ).resolve_legend(color="independent").resolve_scale(x='shared',color="independent") | (bezirke + points+ text).configure_view(stroke=None)

title_karte = alt.Chart(
    {"values": [{"text": "Wähle Radzählstelle aus:"}]}
).mark_text(size=20, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(0)
)
karte=alt.vconcat(
    title_karte,
    (bezirke + points+ text)
)
#.configure_view(
#    stroke=None
#).configure_concat(
#    spacing=1
#)
#title_karte & (bezirke + points+ text)
title_daily_bars = alt.Chart(
    {"values": [{"text": "Tägliche Counts und Temperatur"}]}
).mark_text(size=15, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(0)
)
daily_bars=(title_daily_bars & (chart+holine +holine2 + text_source)& chart_temp
).resolve_legend(color="independent").resolve_scale(x='shared',color="independent")


hconcat_chart=alt.hconcat(
    karte,
    daily_bars
)#.configure_view(stroke=None)

#final_chart & chart_bar


title_monthly_bars = alt.Chart(
    {"values": [{"text": "Counts je Monat"}]}
).mark_text(size=15, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(0)
)

quellen = alt.Chart(
    {"values": [{"text": [" "]}]}
).mark_text(size=10, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(0)
)
quellen1 = alt.Chart(
    {"values": [{"text": ["Quellen: SenUVK, Deutscher Wetterdienst"]}]}
).mark_text(size=10, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(10)
)

final_chart =alt.vconcat(
    hconcat_chart,
    (title_monthly_bars & chart_bar),
    (quellen & quellen1)
).configure_view(
    stroke=None
).configure_concat(
    spacing=4
)

final_chart

alt.VConcatChart(...)

In [607]:
final_chart.save('out_dash.html')

In [648]:
data_temp_hist['month']=data_temp_hist.datetime.dt.month
data_temp_hist['year']=data_temp_hist.datetime.dt.year

In [649]:
data_temp_hist.groupby(['year','month']).TT_TU.mean()

year  month
2002  1         2.842066
      2         5.626884
      3         5.628571
      4         8.787798
      5        15.576944
                 ...    
2018  8        20.962769
      9        16.101250
      10       10.757412
      11        5.303472
      12        4.461962
Name: TT_TU, Length: 204, dtype: float64

In [651]:
source=data_temp_hist.groupby(['year','month']).TT_TU.mean().reset_index()

chart=alt.Chart(source).mark_line().encode(
    x='month:N',
    y='TT_TU:Q',
    color=alt.Color('year:O'),
   # detail='location_parent'
).properties(width=500)
chart

alt.Chart(...)

In [657]:
source=data_temp_hist.groupby(['year','month']).TT_TU.mean().reset_index()

chart=alt.Chart(source).mark_bar().encode(

    x=alt.X('year:N', title='', axis=None),
    y=alt.Y('TT_TU:Q'),
    color=alt.Color('year:O'),
    #detail='location_parent',
    facet=alt.Facet('month:O', spacing =5),
    tooltip=['year','month','TT_TU']
).properties(width=50)

chart

alt.Chart(...)

In [658]:

source=data_temp_hist.groupby(['year','month']).TT_TU.mean().reset_index()
source=source[source.year>2015]

chart=alt.Chart(source).mark_bar().encode(

    x=alt.X('year:N', title='', axis=None),
    y=alt.Y('TT_TU:Q'),
    color=alt.Color('year:O'),
    #detail='location_parent',
    facet=alt.Facet('month:O', spacing =5),
    tooltip=['year','month','TT_TU']
).properties(width=50)

chart

alt.Chart(...)

In [666]:
#https://www.dwd.de/DE/leistungen/klimadatendeutschland/klarchivstunden.html
from zipfile import ZipFile

zip_file = ZipFile(r'wetter/stundenwerte_TU_00403_akt.zip')
data_temp= pd.read_csv(zip_file.open('produkt_tu_stunde_20181005_20200406_00403.txt'), sep=';')

data_temp['datetime'] = pd.to_datetime(data_temp['MESS_DATUM'], format='%Y%m%d%H')

data_temp = data_temp[data_temp['TT_TU']!=-999.].copy()

In [667]:
data_temp_all=data_temp.append(data_temp_hist)

In [668]:
data_temp_all['month']=data_temp_all.datetime.dt.month
data_temp_all['year']=data_temp_all.datetime.dt.year

In [670]:
source=data_temp_all.groupby(['year','month']).TT_TU.mean().reset_index()
source=source[source.year>2015]

chart=alt.Chart(source).mark_bar().encode(

    x=alt.X('year:N', title='', axis=None),
    y=alt.Y('TT_TU:Q'),
    color=alt.Color('year:O'),
    #detail='location_parent',
    facet=alt.Facet('month:O', spacing =5),
    tooltip=['year','month','TT_TU']
).properties(width=50)

chart

alt.Chart(...)

In [ ]:
FINFINALELLAE

In [145]:
source

,location_parent,date,volume
1706,Alberichstraße,2020-03-02,427
1707,Alberichstraße,2020-03-03,415
1708,Alberichstraße,2020-03-04,425
1709,Alberichstraße,2020-03-05,535
1710,Alberichstraße,2020-03-06,307
...,...,...,...
29341,Yorckstraße,2020-04-14,3156
29342,Yorckstraße,2020-04-15,3566
29343,Yorckstraße,2020-04-16,4335
29344,Yorckstraße,2020-04-17,4375


In [166]:

#input
source = day_data_grp.copy()
source['weekday_name']=pd.to_datetime(source.date).dt.weekday_name
source.date=source.date.astype(str)
source['location_parent']=source['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')



source1 = week_data_week10.copy()
source1['location_parent']=source1['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')

source2 = week_data_afterweek10.copy()
source2.xmin=source2.xmin.astype(str)
source2.xmax=source2.xmax.astype(str)
source2['location_parent']=source2['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')

source_temp.date=source_temp.date.astype(str)

#selections
selection = alt.selection_single(fields=['location_parent'], init={'location_parent': 'MariendorferDamm'}, empty='none')
single_nearest = alt.selection_single(on='mouseover', empty='none')

#style
color_bars=alt.Color('location_parent:N', legend=None)



### BARS
chart=alt.Chart(source).mark_bar().encode(
    x=alt.X('date:T', title='',axis=alt.Axis(orient='top')),
    y='volume:Q',
    color=color_bars,
    #tooltip='location_parent'
    tooltip=['location_parent','volume'],#'weekday_name'] #,'date:N'
).transform_filter(
    selection
).add_selection(
    selection
)


holine = alt.Chart(source1).mark_rule(strokeDash=[5,5], color='gray').encode(
    y='volume:Q'
).transform_filter(
    selection
)

holine2 = alt.Chart(source2).mark_rule(color='red').encode(
    y='volume:Q',
    x=alt.X('xmin:T', axis=None),
    x2="xmax:T"
).transform_filter(
    selection
)


### TEMP1
domain = ['current', 'historic']
range_ = ['#507BAA', 'gray']


chart_temp=alt.Chart(source_temp).mark_line().encode(
    x=alt.X('date:T', title=''),
    #y='TT_TU:Q',
    y=alt.Y('TT_TU:Q', title='Temperatur [°C]'),
    color=alt.Color('type_:N', scale=alt.Scale(domain=domain, range=range_)),
).properties(
        height=100,
    )

##### MAP

bezirke=alt.Chart(bezirke_layer).mark_geoshape(stroke='white',strokeWidth=0.5, color='lightgray').encode()


points=alt.Chart(standort_daten_).mark_circle(
    color='black',
    stroke='white',strokeWidth=0.0
).encode(
    longitude='Längengrad:Q',
    latitude='Breitengrad:Q',
    size=alt.condition(single_nearest, alt.value(150), alt.value(70)),
   # text=alt.Text('Beschreibung - Fahrtrichtung:O')
).add_selection(
    selection,single_nearest
).properties(
        height=450,
        width=450,
    )

text=alt.Chart(standort_daten_).mark_text(
    color='black',dy=-10, strokeWidth=0
).encode(
    longitude='Längengrad:Q',
    latitude='Breitengrad:Q',
    text=alt.Text('location_parent:O'),
    fillOpacity=alt.condition(single_nearest, alt.value(1), alt.value(0)),
)

text_source=alt.Chart(standort_daten_).mark_text(
    color='black', dx=-150, dy=-140
).encode(
    text=alt.Text('location_parent:O'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0)),
)#.add_selection(
#    selection#,single_nearest
#)


### BARS LONG and TEMP long
source_bar=day_data.groupby(['month','year','location_parent']).volume.mean().reset_index()

source_bar=source_bar[source_bar.year>2015]
source_bar['location_parent']=source_bar['location_parent'].str.replace('ü','ue').str.replace('ß','ss').str.replace(' ','')

#selection = alt.selection_multi(fields=['location_parent'])#, init={'weekday': 'Monday'})


chart_bar=alt.Chart(source_bar).mark_bar().encode(

    x=alt.X('year:N', title='', axis=None),
    y=alt.Y('volume:Q'),
    color=alt.Color('year:O', legend=None),
    detail='location_parent',
    facet=alt.Facet('month:O', spacing =5),
    tooltip=['location_parent','volume']
).transform_filter(
    selection
).add_selection(
    selection
).properties(width=45)



data_temp_all['year']=data_temp_all.datetime.dt.year

source_temp_all=data_temp_all.groupby(['year','month']).TT_TU.mean().reset_index()
source_temp_all=source_temp_all[source_temp_all.year>2015]

chart_temp_all=alt.Chart(source_temp_all).mark_bar().encode(

    x=alt.X('year:N', title=''),# axis=None),
    y=alt.Y('TT_TU:Q', title='Temperatur [°C]'),
    color=alt.Color('year:O', legend=None),
    #detail='location_parent',
    facet=alt.Facet('month:O', spacing =5, title='',header=alt.Header(title=None, labelFontSize=0)),# header=None),
    tooltip=['year','month','TT_TU']
).properties(width=45, height=120)


#######################################################

title_karte = alt.Chart(
    {"values": [{"text": "Wähle Radzählstelle aus:"}]}
).mark_text(size=20, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(0)
)
karte=alt.vconcat(
    title_karte,
    (bezirke + points+ text+ text_source)
)



title_daily_bars = alt.Chart(
    {"values": [{"text": "Tageswerte und Temperatur der letzten Wochen"}]}
).mark_text(size=15, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(0)
)
daily_bars=(title_daily_bars & (chart+holine +holine2 + text_source)& chart_temp
).resolve_legend(color="independent").resolve_scale(x='shared',color="independent")


hconcat_chart=alt.hconcat(
    karte,
    daily_bars
)#.configure_view(stroke=None)

#final_chart & chart_bar


title_monthly_bars = alt.Chart(
    {"values": [{"text": "Jahresvergleich der Tageswerte je Monat (Radf. pro Tag & Durchschnittstemperatur)"}]}
).mark_text(size=15, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(0)
)

quellen = alt.Chart(
    {"values": [{"text": [" "]}]}
).mark_text(size=10, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(0)
)
quellen1 = alt.Chart(
    {"values": [{"text": ["Quellen: SenUVK, Deutscher Wetterdienst"]}]}
).mark_text(size=10, align='left').encode(  #,anchor='start' , align='left'
    text="text:N",
    x=alt.value(10)
)

final_chart =alt.vconcat(
    hconcat_chart,
    (title_monthly_bars & chart_bar & chart_temp_all),
    (quellen & quellen1)
).configure_view(
    stroke=None
).configure_concat(
    spacing=4
)

final_chart

alt.VConcatChart(...)

In [167]:
final_chart.save('out_corona_dash_v01.html')